
Autor: NeuralNine
Colorize black and white images in Python
https://www.youtube.com/watch?v=oNjQpq8QuAo

Convertido a Jupyter Notebook por Jaime Olvera

Se trabajará con Redes Neuronales Convolucionales

In [1]:
import numpy as np
# cv2 proviene de opencv-python, por lo que se instalará con ese nombre
import cv2

In [2]:
# se cargan archivos auxiliares, estos provienen de la siguiente liga:
# modelo: htps://github/richzhang/colorization/tree/caffe/colorization/models
# points: htps://github/richzhang/colorization/tree/caffe/colorization/resources.pts_in_hull.npy
# inspired: https://github/opencv/opencv/blob/master/samples/dnn/colorization.py
# Los archivos que se baja de ahí se ponen en una carpeta llamada ´model´

prototext_path =  "./model/colorization_deploy_v2.prototxt"
model_path = "./model/colorization_release_v2.caffemodel"
kernel_path = "./model/pts_in_hull.npy"
image_path = "./images/image.jpg"

In [3]:
# se construye la red neural y se llamará ´net´. La red neural ya ha sido entrenada previamiente
net = cv2.dnn.readNetFromCaffe(prototext_path, model_path)

In [4]:
# usando el método ´load´ de numpy se carga la imagen a trabajar y recibe el nombre de ´points´
points = np.load(kernel_path)

In [5]:
# A la imagen llamada points se le hace traspuesta y se redifine su tamaño, luego se le pasa a la red neural
# para entender mejor esta parte es importante revisar la sintaxis de opencv

points = points.transpose().reshape(2,313,1,1)
net.getLayer(net.getLayerId("class8_ab")).blobs = [points.astype(np.float32)]
net.getLayer(net.getLayerId("conv8_313_rh")).blobs = [np.full([1,313],  2.606, dtype="float32")]

In [6]:
#Ahora se carga la imagen blanco y negro, se normaliza y luego se convierte a BGR
bw_image = cv2.imread(image_path)
normalized = np.float32(bw_image) /255.0
lab = cv2.cvtColor(normalized, cv2.COLOR_BGR2LAB)

In [7]:
# La imagen se reajusta de tamaño y luego se le hace un split (se divide) del color [0]
resized = cv2.resize(lab, (224,224))
L = cv2.split(resized)[0]
L -= 50

In [8]:
# Enseguida de toma el split L y se le pasa a la red neuronal . Se le aplica traspuesta
net.setInput(cv2.dnn.blobFromImage(L))
ab = net.forward()[0, :, :, :].transpose((1,2,0))

In [9]:
# La red se reajusta al tamaño de la imagen blanco y negro y nuevamente se hace split
ab = cv2.resize(ab,(bw_image.shape[1], bw_image.shape[0]))
L = cv2.split(lab)[0]

#L es la brillantez ( lighness)

In [10]:
# ahora se mezcla la brillantes con los colores Rojo, Verde y Azul (RGB)
colorized = np.concatenate((L[:, :, np.newaxis],ab),axis =2)
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2RGB)
colorized = (255.0 * colorized).astype("uint8")
colorized = cv2.resize(colorized,(int(bw_image.shape[1]*0.1), (int(bw_image.shape[0]*0.1))))
# se produce la imagen a color

In [11]:
# se muestran las imágenes en blanco y negro así como la coloreada. Luego se
# destruyen los objetos. Habría que buscar la forma de guardar la imagen a color
#cv2.imshow("BW Image", bw_image)
cv2.imshow("Colorizes",colorized)
cv2.waitKey(0)
cv2.destroyAllWindows()